In [1]:
from brian2 import *
import numpy as np
import warnings

#warnings.filterwarnings("ignore", category=UserWarning, module='brian2.codegen.generators.base')

start_scope()

defaultclock.dt = 0.0001*ms  

# Custom timing function
@implementation('numpy', discard_units=True)
@check_units(w=1, global_clock=1, layer=1, result=1, sum=1, spikes_received=1)
def spike_timing(w, global_clock, layer, sum, spikes_received): 
    #print(global_clock)
    x = global_clock % 1
    if w >= 0:
        return (x ** (1 - w)) 
    else:
        return (1 - (1 - x) ** (1 + w)) 
    
@implementation('numpy', discard_units=True)
@check_units(layer=1, result=1, sum=1, spikes_received=1)
def math1(layer, sum, spikes_received): 
    return (sum/spikes_received )+ layer


In [2]:
# Urd, Verdande, Skuld 

# did not apply the specficyed weifhts to the first layer yet will maybe do after
def run_Urd(inputs, weights_1, weights_2, weights_3):
    '''4-10-3 SNN'''
    # will add check of weights # so it all works
    n_input = 4 
    n_hidden = 10
    n_output = 3
    n_total = n_input + n_hidden + n_output

    neurons = NeuronGroup(n_total, '''
        v : 1
        sum : 1
        spikes_received : 1
        scheduled_time : second
        global_clock : 1
    ''', threshold='v > 1', reset='v = 0', method='exact')
    neurons.v = 0
    neurons.scheduled_time = 1e9 * second
    neurons.global_clock = 0.0
    neurons.sum = 0.0
    neurons.spikes_received = 0.0


    indicess = [i for i in range(n_input)]
    stim = SpikeGeneratorGroup(n_input, indices=indicess, times=(inputs*ms))

    syn_input = Synapses(stim, neurons[0:n_input], '''
        w : 1
        layer : 1
    ''', on_pre='''
        spikes_received += 1
        sum += spike_timing(w, global_clock, layer, spikes_received, sum)
        scheduled_time = ((sum/spikes_received) + layer) * ms 
    ''')
    syn_input.connect(j='i')
    syn_input.w = weights_1
    syn_input.layer = 0

    syn_hidden = Synapses(neurons[0:n_input], neurons[n_input:n_input+n_hidden], '''
        w : 1
        layer : 1
    ''', on_pre='''
        spikes_received += 1
        sum += spike_timing(w, global_clock, layer, spikes_received, sum)
        scheduled_time = ((sum/spikes_received) + layer) * ms 
    ''')
    for inp in range(n_input):
        for hid in range(n_hidden):
            syn_hidden.connect(i=inp, j=hid)

    syn_hidden.w = weights_2
    syn_hidden.layer = 1


    syn_output = Synapses(
        neurons[n_input:n_input+n_hidden], 
        neurons[n_input+n_hidden:n_total], 
        '''
        w : 1
        layer : 1
        ''',
        on_pre='''
        spikes_received += 1
        sum += spike_timing(w, global_clock, layer, spikes_received, sum)
        scheduled_time = ((sum/spikes_received) + layer) * ms 
        '''
    )

    for hid in range(n_hidden):
        for out in range(n_output):
            syn_output.connect(i=hid, j=out)

    # Set weights in correct order
    syn_output.w[:] = weights_3
    syn_output.layer = 2

    # print(syn_output.i[:], syn_output.j[:])
    # weights_into_output_1 = weights_3[1::3]



    neurons.run_regularly('''
        v = int(abs(t - scheduled_time) < 0.0005*ms) * 1.2
        global_clock += 0.001
    ''', dt=0.001*ms)


    spikemon = SpikeMonitor(neurons)

    run(5*ms)

    result = []

    for i in range(n_total):
        times = spikemon.spike_trains()[i]
        if len(times) > 0:
            result.append(round(times[0]/ms, 3))
        else:
            result.append(None)  # or some other placeholder like float('nan')
            
    return result


def calc_cost(outputs, desired_outputs):
    return 0.5 * ((outputs - desired_outputs) ** 2)


w_1 = [0] * 4 #np.random.uniform(  0.05, .95, size=4) #[0.5] * 4
w_2 = [0] * 40 #np.random.uniform( -.95, .95, size=40) #[0.5] * 40
w_3 = [0] * 30 #np.random.uniform( -.95, .95, size=30) #[0.5] * 30 



def update_weights_input_neuron(hidden_activations, weights, actual_output, desired_output, learning_rate=0.1):

    """
    Update all weights going into a single output neuron using gradient descent.

    Args:
        hidden_activations (np.ndarray): Activations from hidden neurons (shape: [n_hidden]).
        weights (np.ndarray): Current weights into the output neuron (shape: [n_hidden]).
        actual_output (float): Current output of this neuron.
        desired_output (float): Target output for this neuron.
        learning_rate (float): Learning rate.

    Returns:
        np.ndarray: Updated weights (shape: [n_hidden]).
    """
    error = actual_output - desired_output
    gradients = error * hidden_activations
    updated_weights = weights - learning_rate * gradients
    updated_weights = np.clip(updated_weights, -0.999999, 0.999999)
    return updated_weights


def update_last_layer_weights(spike_times, desired_outputs):

    lr = 0.2 # will make more global later
    
    var = spike_times
    hidden_activations = np.array(var[4:14])


    weights_to_output_0 = w_3[0::3]
    weights_to_output_1 = w_3[1::3]
    weights_to_output_2 = w_3[2::3]

    actual_output_0 = var[-3]
    actual_output_1 = var[-2]
    actual_output_2 = var[-1]


    desired_output_0 = desired_outputs[0]
    desired_output_1 = desired_outputs[1]
    desired_output_2 = desired_outputs[2]


    new_weights_0 = update_weights_input_neuron(
        hidden_activations,
        weights_to_output_0,
        actual_output_0,
        desired_output_0,
        learning_rate=lr
    )

    new_weights_1 = update_weights_input_neuron(
        hidden_activations,
        weights_to_output_1,
        actual_output_1,
        desired_output_1,
        learning_rate=lr
    )

    new_weights_2 = update_weights_input_neuron(
        hidden_activations,
        weights_to_output_2,
        actual_output_2,
        desired_output_2,
        learning_rate=lr
    )

    w_3[0::3] = new_weights_0
    w_3[1::3] = new_weights_1
    w_3[2::3] = new_weights_2

    print("weights updated")



In [3]:
inputs = [0.5] * 4 #np.random.uniform(0.1, .95, size=4) #[0.5]* 4
#desired_outputs = np.array([.9] * 3) # np.random.uniform(0.1, .95, size=3)
wanted_output = [2.1, 2.5, 2.9]

# w_1 = [0] * 4 #np.random.uniform(  0.05, .95, size=4) #[0.5] * 4
# w_2 = [0] * 40 #np.random.uniform( -.95, .95, size=40) #[0.5] * 40
# w_3 = [0] * 30 #np.random.uniform( -.95, .95, size=30) #[0.5] * 30

#var = run_Urd(inputs, w_1, w_2, w_3)

def real_outputs(var):
    real_output = [-1]*3
    for i in range(3):
        real_output[i] = round(var[-3+i] - 2, 4)
    return(real_output)



import logging

logging.getLogger('brian2').setLevel(logging.ERROR)

for i in range(5):
    try:
        var = run_Urd(inputs, w_1, w_2, w_3)
        print(real_outputs(var))
        update_last_layer_weights(var, wanted_output)
    except Exception as e:
        print(f"Error in iteration {i}: {e}")

print(real_outputs(var))






# for i in range(5):
#     var = run_Urd(inputs, w_1, w_2, w_3)
#     print(real_outputs(var))
#     update_last_layer_weights(var, wanted_output)
# print(real_outputs(var))



[0.503, 0.503, 0.503]
weights updated
[0.459, 0.503, 0.546]
weights updated
[0.417, 0.502, 0.587]
weights updated
[0.377, 0.502, 0.627]
weights updated
[0.339, 0.502, 0.663]
weights updated
[0.339, 0.502, 0.663]


In [4]:
list = [i for i in range(5)]
print(list)
print(list[-3])

[0, 1, 2, 3, 4]
2
